## Rigid body verification

In [ ]:
import warnings
#warnings.filterwarnings('ignore')
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
from seaman.systems.cppsystems.bodies import RigidBody
from seaman.systems.python.force import Gravity,Force
from seaman.systems.cppsystems.forcegen import Frame
from pysim.simulation import Sim

import rigidbody.rigid_body_dynamics as rbd
from rigidbody.plotting import TrackPlot3dWidget,track_plot
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
def do_seaman_simulation(t, force_torque = [0, 0, 0, 0, 0, 0], initial_speeds = [0, 0, 0, 0, 0, 0],
                   initial_coordinates = [0, 0, 0, 0, 0, 0], mass = 1, I_xx = 1, I_yy = 1, I_zz = 1):
    
    point = RigidBody()
    point.inputs.mass = mass
    r_xx = np.sqrt(I_xx / mass)
    r_yy = np.sqrt(I_yy / mass)
    r_zz = np.sqrt(I_zz / mass)
    point.inputs.gyradii = [r_xx, r_yy, r_zz]

    f1 = Frame()
    force = Force()
    r12 = np.array([0, 0, 0])
    f1.add_subsystem(force, "force", r12)
    point.add_subsystem(f1, 'force')

    force.inputs.force = force_torque
    point.states.position_world = initial_coordinates[0:3]
    point.states.attitude_world = initial_coordinates[3:]
    point.states.linear_velocity = initial_speeds[0:3]
    point.inputs.angular_velocity = initial_speeds[3:]

    point.store_all()

    sim = Sim()
    sim.add_system(point)

    sim.simulate(t[-1], t[1] - t[0])
    position = point.res.position_world
    attitude = point.res.attitude_world
    
    
    df = pd.DataFrame(index = point.res.time)
    df['x0'] = position[:,0]
    df['y0'] = position[:,1]
    df['z0'] = position[:,2]
    df['phi'] = attitude[:,0]
    df['theta'] = attitude[:,1]
    df['psi'] = attitude[:,2]
    
        
    return df



## Circle simulation

In [ ]:
radius = 10  # Radius of rotation [m]
w = 0.1  # Angle velocity [rad/s]
V = radius*w  # Speed of point [m/s]
t = np.linspace(0,2*np.pi/w,100)

mass = 1
I_xx = 1
I_yy = 1
I_zz = 1

expected_acceleration = -radius*w**2
expected_force = mass*expected_acceleration

force_torque = [0,-expected_force,0,0,0,0]
initial_speeds = [V,0,0,0,0,w]
initial_coordinates = [0, -radius, 0,0,0,0]


df = do_seaman_simulation(t=t, force_torque=force_torque, initial_speeds=initial_speeds, 
                             initial_coordinates=initial_coordinates, mass=mass, I_xx=I_xx, I_yy=I_yy, I_zz=I_zz)

fig,ax = plt.subplots()
track_plot(df,ax, time_step = '5S')


fig,ax = plt.subplots()
df.plot(y='psi', ax=ax);
ax.set_xlabel('time')
ax.set_ylabel('psi')


## Yaw rate at 45 deg roll
This one gives different results.

In [ ]:
t = np.linspace(0,2*np.pi/w,100)

mass = 1
I_xx = 10
I_yy = 10
I_zz = 10

t = np.linspace(0, 10, 100)


initial_speeds = [0,0,0,0.0,0.0,0.3]

initial_coordinates = [0, 0, 0, np.deg2rad(45), 0, 0]

force_torque = [0,0,0,0,0,0]

df = do_seaman_simulation(t=t, initial_speeds=initial_speeds, 
                          initial_coordinates=initial_coordinates,
                         mass=mass,I_xx=I_xx, I_yy=I_yy, I_zz=I_zz)

# Run an alternative simulation:
df_alternative = rbd.simulate(t=t, initial_speeds=initial_speeds, initial_coordinates=initial_coordinates,
                             mass=mass,I_xx=I_xx, I_yy=I_yy, I_zz=I_zz,force_torque=force_torque)

fig,ax = plt.subplots()
track_plot(df,ax, time_step = '5S', style='r-', label = 'seaman')
track_plot(df_alternative,ax, time_step = '5S',style='b:', label = 'alt')

fig,ax = plt.subplots()
df.plot(y='phi', ax=ax, label='seaman', style='r-');
df_alternative.plot(y='phi', ax=ax, label='alt', style='b:')
ax.set_xlabel('time')
ax.set_ylabel('phi')

fig,ax = plt.subplots()
df.plot(y='theta', ax=ax, label='seaman', style='r-');
df_alternative.plot(y='theta', ax=ax, label='alt', style='b:')
ax.set_xlabel('time')
ax.set_ylabel('theta')

fig,ax = plt.subplots()
df.plot(y='psi', ax=ax, label='seaman', style='r-');
df_alternative.plot(y='psi', ax=ax, label='alt', style='b:')
ax.set_xlabel('time')
ax.set_ylabel('psi')



In [ ]:
scene = TrackPlot3dWidget(df=df)

In [ ]:
scene = TrackPlot3dWidget(df=df_alternative)

In [ ]:
t = np.linspace(0,2*np.pi/w,100)

mass = 1
I_xx = 10
I_yy = 10
I_zz = 10

t = np.linspace(0, 10, 100)


initial_speeds = [0,0,0,0,0.3,0]

initial_coordinates = [0, 0, 0, np.deg2rad(89), 0, 0]

force_torque = [0,0,0,0,0,0]

df = do_seaman_simulation(t=t, initial_speeds=initial_speeds, 
                          initial_coordinates=initial_coordinates,
                         mass=mass,I_xx=I_xx, I_yy=I_yy, I_zz=I_zz)

# Run an alternative simulation:
df_alternative = rbd.simulate(t=t, initial_speeds=initial_speeds, initial_coordinates=initial_coordinates,
                             mass=mass,I_xx=I_xx, I_yy=I_yy, I_zz=I_zz,force_torque=force_torque)

fig,ax = plt.subplots()
track_plot(df,ax, time_step = '5S', style='r-', label = 'seaman')
track_plot(df_alternative,ax, time_step = '5S',style='b:', label = 'alt')

fig,ax = plt.subplots()
df.plot(y='phi', ax=ax, label='seaman', style='r-');
df_alternative.plot(y='phi', ax=ax, label='alt', style='b:')
ax.set_xlabel('time')
ax.set_ylabel('phi')

fig,ax = plt.subplots()
df.plot(y='theta', ax=ax, label='seaman', style='r-');
df_alternative.plot(y='theta', ax=ax, label='alt', style='b:')
ax.set_xlabel('time')
ax.set_ylabel('theta')

fig,ax = plt.subplots()
df.plot(y='psi', ax=ax, label='seaman', style='r-');
df_alternative.plot(y='psi', ax=ax, label='alt', style='b:')
ax.set_xlabel('time')
ax.set_ylabel('psi')

In [ ]:
scene = TrackPlot3dWidget(df=df)

In [ ]:
scene = TrackPlot3dWidget(df=df_alternative)

In [ ]:
mass = 1
I_xx = 10
I_yy = 10
I_zz = 10

t = np.linspace(0, 10, 1000)


force_torque = [0.3730018553631673, 0.6692513437947458, -0.9634234453116164, 0.5002886298899349, 0.9777221778129894, 0.4963313087596788]

df = do_seaman_simulation(t=t, force_torque=force_torque,
                         mass=mass,I_xx=I_xx, I_yy=I_yy, I_zz=I_zz)

# Run an alternative simulation:
df_alternative = rbd.simulate(t=t, mass=mass,I_xx=I_xx, I_yy=I_yy, I_zz=I_zz,force_torque=force_torque)

fig,ax = plt.subplots()
track_plot(df,ax, time_step = '5S', style='r-', label = 'seaman')
track_plot(df_alternative,ax, time_step = '5S',style='b:', label = 'alt')

fig,ax = plt.subplots()
df.plot(y='phi', ax=ax, label='seaman', style='r-');
df_alternative.plot(y='phi', ax=ax, label='alt', style='b:')
ax.set_xlabel('time')
ax.set_ylabel('phi')

fig,ax = plt.subplots()
df.plot(y='theta', ax=ax, label='seaman', style='r-');
df_alternative.plot(y='theta', ax=ax, label='alt', style='b:')
ax.set_xlabel('time')
ax.set_ylabel('theta')

fig,ax = plt.subplots()
df.plot(y='psi', ax=ax, label='seaman', style='r-');
df_alternative.plot(y='psi', ax=ax, label='alt', style='b:')
ax.set_xlabel('time')
ax.set_ylabel('psi')

In [ ]:
scene = TrackPlot3dWidget(df=df)

In [ ]:
scene = TrackPlot3dWidget(df=df_alternative)